# Test Task for Code Reviewer

Time given: 1 day

In [1]:
import pandas as pd
import numpy as np
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

# Task 1. Working with data

To complete this task, use the data set in the attached file. Indicate the answer to each of the
following steps and time to complete the entire task.



In [2]:
# 1.1. Download the data set movie_metadata.csv, which contains data about films from IMDb (Internet Movie Database).
movie_df = pd.read_csv('movie_metadata.csv')
print(movie_df.shape)
movie_df.head(3)

(5043, 28)


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0$,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0$,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0$,2015.0,393.0,6.8,2.35,85000


In [3]:
# 1.2. The duration column contains data on the film length. How many missing values are there in this column?
movie_df.duration.isna().sum()

15

In [4]:
# 1.3. Replace the missing values in the duration column with the median value for this column.
movie_df.duration.fillna(movie_df.duration.median(), inplace=True)

In [5]:
# 1.4. What is the average film length? Give the answer as a floating-point figure rounded to two decimal places.
round(movie_df.duration.mean(), 2)

107.19

In [6]:
# 1.5. Create a movie_duration_category column, which will contain three categories depending on the film length:
# • Category "1. <90" if the film is less than 90 minutes long
# • Category "2. 90–120" if the film is between 90 minutes and two hours long (inclusively)
# • Category "3. >120" if the film is more than two hours long
categories = {
    "1. <90": movie_df.duration < 90, 
    "2. 90–120": (movie_df.duration >= 90) & (movie_df.duration <= 120),
    "3. >120": movie_df.duration > 120
}
movie_df['movie_duration_category'] = np.select(categories.values(), categories.keys())

In [7]:
# 1.6. Build a summary table for films released after 2000 (inclusively), to list the numbers of films:
# • Table rows: year
# • Table columns: movie duration category ("<90", "90–120", ">120")
# • The year of release should be displayed in the YYYY format.
summary_df = movie_df[movie_df.title_year >= 2000] \
                .drop_duplicates() \
                .groupby(['title_year', 'movie_duration_category']) \
                .size() \
                .unstack(level=-1)
summary_df.index = summary_df.index.astype(int)
summary_df

movie_duration_category,1. <90,2. 90–120,3. >120
title_year,,,
2000,25,110,34
2001,29,118,38
2002,33,145,27
2003,31,108,30
2004,30,140,41
2005,31,142,48
2006,40,145,53
2007,29,130,43
2008,29,160,36


In [8]:
# 1.7. How many films between 90 minutes and two hours long were released in 2008?
summary_df.loc[2008]["2. 90–120"]

160

In [9]:
# 1.8. The plot_keywords column holds keywords characterizing the film's plot. Using the data
# in this column, create a column called movie_plot_category, to contain four categories
# depending on the key words in the column:
# • Category "love_and_death" if the keywords include both "love" and "death"
# • Category "love" if the keywords include the word "love"
# • Category "death" if the keywords include the word "death"
# • Category "other" if the keywords do not meet the conditions above
f = lambda x: movie_df.plot_keywords.astype(str).str.contains(x)
# '(?=.*love)(?=.*death)'

categories = {
    "love_and_death": (f('love')) & (f('death')), 
    "love": f('love'),
    "death": f('death')
}
movie_df['movie_plot_category'] = \
    np.select(categories.values(), categories.keys(), default="other")

In [10]:
# 1.9. The imdb_score column shows a viewer rating for the film. Build a table to reflect the
# average rating of films depending on which movie_plot_category category they belong to.

ratings_df = movie_df.groupby('movie_plot_category').imdb_score.agg('mean').to_frame('average_imdb_score')
ratings_df

,average_imdb_score
movie_plot_category,
death,6.535465
love,6.580769
love_and_death,6.506250
other,6.431422


In [11]:
# 1.10. What is the average rating of films in the "love" category? Give the answer as a floatingpoint
# figure rounded to two decimal places.
ratings_df.loc['love'].round(2).average_imdb_score

6.58

In [12]:
# 1.11. The budget column contains the film's budget. What is the median budget for all the films
# listed? Give the answer as an integer.
int(movie_df.budget.apply(lambda x: float((x.split('$')[0]))).median())

15000000

# Task 2. Problem-solving

To complete this task, use the data set in the attached file. Indicate the answer to each of the
following steps and time to complete the entire task.


In [13]:
# 1. Download the event_data.csv dataset, which contains data on the use of the mobile
# application of users who registered from July 29 to September 1, 2019:
event_df = pd.read_csv('event_data.csv')
print(event_df.shape)
event_df.head(3)

(79742, 4)


,user_id,event_date,event_type,purchase_amount
0,c40e6a,2019-07-29 00:02:15,registration,NaN
1,a2b682,2019-07-29 00:04:46,registration,NaN
2,9ac888,2019-07-29 00:13:22,registration,NaN


In [14]:
# 2. Highlight user cohorts based on the week of registration in the application. The cohort
# identifier should be the week ordinal (for example, the week from July 29 to August 4
# should have identifier 31).

# event_df.groupby('event_type').user_id.agg('nunique')

event_df['event_date'] = pd.to_datetime(event_df.event_date, format='%Y-%m-%d %H:%M:%S')

reg_df = event_df[event_df.event_type=='registration'][['user_id', 'event_date']]
reg_df['cohort_id'] = event_df.event_date.dt.isocalendar().week

# map the right cohort_id to all event_types
di = dict(zip(reg_df.user_id, reg_df.cohort_id))
event_df['cohort_id'] = event_df.user_id.replace(di)
event_df.head(2)

,user_id,event_date,event_type,purchase_amount,cohort_id
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31
1,a2b682,2019-07-29 00:04:46,registration,NaN,31


In [15]:
# 3. How many unique users in the cohort with ID 33?
event_df[event_df.cohort_id==33].user_id.nunique()

2045

In [16]:
# 4. For each event, highlight the indicator lifetime - the weekly lifetime of the cohort. The
# lifetime indicator is calculated based on the serial number of the week in which the event
# is committed, relative to the week of registration. For example, an event committed on
# August 3 by a user from a cohort of registrants at 31 weeks will be committed on the zero
# week of lifetime, and an event committed by the same user on August 5 will be committed
# on the first week of lifetime).

event_df['weekly_lifetime'] = event_df.event_date.dt.isocalendar().week - event_df.cohort_id


In [17]:
# 5. Build a summary table of changes in the Retention Rate for cohorts depending on lifetime.
def calc_retention_rate(x):
    return round(100 * x.groupby('weekly_lifetime').size() / x.weekly_lifetime.size, 2)

retention_df = event_df.groupby('cohort_id').apply(calc_retention_rate).to_frame('retention_rate').unstack(level=-1)
retention_df

retention_rate                           
weekly_lifetime              0      1      2      3     4
cohort_id                                                
31                       27.26  35.22  22.12  11.43  3.97
32                       28.07  36.66  23.24  12.04   NaN
33                       31.74  41.19  27.06    NaN   NaN
34                       43.41  56.59    NaN    NaN   NaN
35                      100.00    NaN    NaN    NaN   NaN

In [18]:
# 6. What is the 3 week retention rate for a cohort with ID 32? Give the answer in percent,
# rounded to 2 decimal places, inclusive.
retention_df.loc[32][3]

12.04

In [19]:
# 7. Build a summary table of changes in the indicator ARPPU (Average Revenue Per Paying
# User) for cohorts depending on lifetime.

def calc_arppu(x):
    return round(x.groupby('weekly_lifetime').purchase_amount.sum() / x.user_id.nunique(),2)

arppu_df = event_df.groupby('cohort_id').apply(calc_arppu).to_frame('arppu').unstack(level=-1)
arppu_df


arppu                         
weekly_lifetime     0      1     2     3     4
cohort_id                                     
31               4.50  10.40  6.18  3.10  1.02
32               5.56  10.78  6.45  3.21   NaN
33               4.79  10.63  6.72   NaN   NaN
34               4.82   9.95   NaN   NaN   NaN
35               5.60    NaN   NaN   NaN   NaN

In [20]:
# 8. What is the 3-week ARPPU of a cohort with ID 31? Give the answer with a floating point
# number, rounded to 2 decimal places, inclusive.
arppu_df.loc[31][3]

3.1

In [21]:
# 9. What is the median time between user registration and first purchase? Give the answer
# in seconds (!) As an integer.

# NOTE: This ignores NaT values i.e. those who did not make first purchase yet when calculating the median
temp_df = reg_df.set_index('user_id').join(
    event_df[event_df.event_type == 'purchase'] \
        .groupby('user_id').event_date.min().to_frame('first_purchase_date'))
int((temp_df.first_purchase_date - temp_df.event_date).median() / pd.Timedelta(1, 'S'))

434774

# Task 3. Answering student questions

How would you answer the student's question below? Your task is to get your message across in such a way that a beginner can understand your explanation. You can do this any way you want (pictures, GIFs, metaphors, anything) so long as it makes your explanation clear.
Answer the question: "What is the difference between DataFrame and Series?"
Indicate how much time you spent completing this task.


Answer: 

Before we explain the difference between a DataFrame and a Series, let us first recall the concept of an array in computer science. In simplest terms, an array is a collection of elements. Think of a bunch of kids flanking in one line to buy a sundae at an ice cream stand. In this case, the elements are the kids and the line is the array. Now, one important property of an array is that it is indexed in an ordered manner to indicate the position of its elements. Think of our line of kids. The kid at the front who's currently being served by the stand is indexed "first" in line, the kid behind him is indexed "second" in line and so on. The index is very important because it would be chaotic if the line is disorganized and the ice cream stand wouldn't know who to serve first! One last point about an array: we can name or label it. In case of our array of kids, we can call them "ice cream kids".

Let's get down to the difference between a DataFrame and a Series. In simplest terms, a Series is a one-dimensional (as in "one line") labeled array like our 'ice cream kids'! Now, imagine after the ice cream kids bought their sundae, they flank in one line at the burger stand beside the ice cream stand: we get another Series which we can call "burger kids"! In simplest terms, a DataFrame is a collection of Series i.e. is a two-dimensional (think of rows of kids and columns of stands) labeled array.

# Task 4.

Problem: You are given two random variables X and Y. <br>
E(X) = 0.5, Var(X) = 2 <br>
E(Y) = 7, Var(Y) = 3.5 <br>
cov (X, Y) = -0.8 <br>
Find the variance of the random variable Z = 2X - 3Y

[Theorem](https://statproofbook.github.io/P/var-lincomb.html): The variance of the linear combination of two random variables is a function of the variances as well as the covariance of those random variables:

$ Var(aX+bY) = a^2Var(X)+b^2Var(Y) + 2abCov(X,Y)$

Substituting: 

$Var(Z) = Var(2X - 3Y) = 2^2(2)+(-3)^2(3.5) + 2(2)(-3)(-0.8) = 49.1$

# Task 5.

Omer trained a linear regression model and tested its performance on a test sample of 500
objects. On 400 of those, the model returned a prediction higher than expected by 0.5, and on
the remaining 100, the model returned a prediction lower than expected by 0.7.
What is the MSE for his model?
Limor claims that the linear regression model wasn't trained correctly, and we can do improve
it by changing all the answers by a constant value. What will be her MSE?
You can assume that Limor found the smallest error under her constraints.
Return two values - Omer's and Limor's MSE.


My Solution:


$ (eq. 1)\hspace{1cm} MSE = E[(Y−\hat{Y})^2]$ <br>
$ (eq. 2)\hspace{1cm}MSE = \frac{1}{n} \sum_{i=1}^{n}(Y_i-\hat{Y_i})^2 $<br>
$ (eq. 3)\hspace{1cm}MSE = Var(Y) + Bias(Y, \hat{Y})^2 $<br>
$ (eq. 4)\hspace{1cm}Var(Y) = E[(Y-E[Y])^2]$<br>
$ (eq. 5)\hspace{1cm}Bias(Y, \hat{Y})^2 = (\frac{1}{n}\sum_{i=1}^{n}(Y_i-\hat{Y_i}))^2 $<br>

$ Y$ − actual values, $\hat{Y}$ − predicted values

### **Omer:**
Using $(eq. 2)$

$ MSE = \frac{1}{500} (\sum_{1}^{400}(-0.5)^2 + \sum_{401}^{500}(0.7)^2) = \frac{149}{500} = 0.298$  

### **Limor:**

Calculate the constant value that minimizes the MSE: <br>
Using $(eq. 1)$ and replacing $\hat{Y}$ with $a$ we get: <br>
$ MSE=E[(Y−a)^2] = EY^2−2aEY+a^2.$

Differentiating w.r.t $a$ and equating to 0, we get: <br>
$ −2EY+2a = 0$ <br>
$ a = EY$ 

Substituting the value of $a$ to $(eq. 1)$, we get: <br>
$ MSE = E[(Y-E[Y])^2] $

Refering to $(eq. 4)$, we get: <br>
$MSE = Var(Y)$. 

Refering to $(eq. 3)$, this means that the bias is zero. <br>
Since Limor will not retrain the model and just change the predictions, the model variance remains the same. <br>
This means that the $Var(Y)$ of Omer's model is equal to Limor's $MSE$ i.e.:

$ Limor's \hspace{0.1cm}MSE = Omer's \hspace{0.1cm}Var(Y) = Omer's \hspace{0.1cm}MSE - Omer's \hspace{0.1cm}Bias(Y, \hat{Y})^2 $ <br>
$ Omer's \hspace{0.1cm}Bias(Y, \hat{Y})^2 = ((400*-0.5 + 100*0.7)/500)^2 = 0.0676 $ <br>
$ Limor's \hspace{0.1cm}MSE = 0.298 - 0.0676 = 0.2304$


### **Final Answer**

$MSE_{omer} = 0.298 $ <br>
$MSE_{limor} = 0.2304$